<a href="https://colab.research.google.com/github/JeKo2022/Safeguarding_Eval_Red-teaming/blob/main/CIFAR_FID_Scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pythorch FID: https://pytorch.org/ignite/generated/ignite.metrics.FID.html


The Fréchet inception distance (FID) is a metric used to assess the quality of images created by a generative model, like a generative adversarial network (GAN). (source: https://en.wikipedia.org/wiki/Fr%C3%A9chet_inception_distance,
Sept 2024).
This Notebook uses CIFAR dataset and is merely to illustrate how FID works (normally one would compare real and fake images).  

In [24]:

!pip install torch torchvision torchaudio
!pip install pytorch-fid

In [6]:

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import shutil
from pytorch_fid import fid_score

In [11]:
# Step 3: Download CIFAR-10 dataset and create sample folders
#convert image to tensors for resize and processing after
transform = transforms.Compose([
    transforms.Resize(299),  # Inception expects 299x299 images
    transforms.ToTensor(),
])

# Download CIFAR-10 dataset
cifar10 = datasets.CIFAR10(root='./data', download=True, transform=transform)

# Create directories for sample images
os.makedirs('real_images', exist_ok=True)
os.makedirs('generated_images', exist_ok=True)

Files already downloaded and verified


Test: Images are the same so FID score should be close to zero.

In [13]:
# Copy a small subset of images to 'real_images' and 'generated_images' folders
for i in range(100):  # Adjust the number of images as needed
    img_tensor, _ = cifar10[i]  # Get the image tensor and ignore the label
    img = transforms.ToPILImage()(img_tensor)  # Convert tensor to PIL Image
    img.save(f'real_images/img_{i}.png')
    img.save(f'generated_images/img_{i}.png')

In [14]:
# Step 4: Calculate FID score between the real and generated images
fid_value = fid_score.calculate_fid_given_paths(['real_images', 'generated_images'],
                                                batch_size=50,
                                                device='cuda' if torch.cuda.is_available() else 'cpu',
                                                dims=2048)

100%|██████████| 2/2 [00:00<00:00,  2.49it/s]


In [15]:
print(f'FID Score: {fid_value}')

FID Score: -6.65242810100608e-05


TEST: applying a simple transformation but images are still the same - score should go up.

In [16]:
# Copy a small subset of images to 'real_images' and 'generated_images' folders
for i in range(100):  # Adjust the number of images as needed
    img_tensor, _ = cifar10[i]  # Get the image tensor and ignore the label
    img = transforms.ToPILImage()(img_tensor)  # Convert tensor to PIL Image

    # Save the original image to the 'real_images' folder
    img.save(f'real_images/img_{i}.png')

    # Apply a transformation to create a slightly different image for 'generated_images'
    transformed_img = transforms.functional.hflip(img)  # Flip the image horizontally

    # Save the transformed image to the 'generated_images' folder
    transformed_img.save(f'generated_images/img_{i}.png')

In [17]:
# Step 4: Calculate FID score between the real and generated images
fid_value = fid_score.calculate_fid_given_paths(['real_images', 'generated_images'],
                                                batch_size=50,
                                                device='cuda' if torch.cuda.is_available() else 'cpu',
                                                dims=2048)
print(f'FID Score: {fid_value}')

100%|██████████| 2/2 [00:00<00:00,  2.50it/s]


FID Score: 34.09555126536452


Random images, FID score should be high.

In [19]:
import random

In [20]:


# Copy a small subset of random images to 'real_images' and 'generated_images' folders
real_indices = random.sample(range(len(cifar10)), 100)  # Randomly select 100 indices for real_images
generated_indices = random.sample(range(len(cifar10)), 100)  # Randomly select 100 different indices for generated_images

for i in range(100):
    # Get a random image tensor for real images
    real_img_tensor, _ = cifar10[real_indices[i]]
    real_img = transforms.ToPILImage()(real_img_tensor)  # Convert tensor to PIL Image
    real_img.save(f'real_images/img_{i}.png')

    # Get a different random image tensor for generated images
    generated_img_tensor, _ = cifar10[generated_indices[i]]
    generated_img = transforms.ToPILImage()(generated_img_tensor)  # Convert tensor to PIL Image
    generated_img.save(f'generated_images/img_{i}.png')


In [21]:
# Step 4: Calculate FID score between the real and generated images
fid_value = fid_score.calculate_fid_given_paths(['real_images', 'generated_images'],
                                                batch_size=50,
                                                device='cuda' if torch.cuda.is_available() else 'cpu',
                                                dims=2048)
print(f'FID Score: {fid_value}')

100%|██████████| 2/2 [00:00<00:00,  2.28it/s]


FID Score: 168.09394773862795
